In [ ]:
import pandas as pd
from gensim.models import ldamodel #I could not find the eta-parameter in Mallet :(
import spacy

#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python
#https://markroxor.github.io/gensim/static/notebooks/lda_training_tips.html

def preprocess_corpus(corpus):
    return_object=dict()#or probably something else
    for doc in corpus:
        doc=spacy.part_of_speech_tag(doc)
        doc=spacy.lemmatize(doc)
        doc=spacy.remove_stopwords(doc)
        doc=spacy.make_bigrams(doc)
        return_object[doc]=doc
    return return_object

given_corpus=pd.load(texts)

processed_corpus=preprocess_corpus(given_corpus)
taskpriors=[preprocess_corpus(pd.load(texts_prior1)),preprocess_corpus(pd.load(texts_prior2)),preprocess_corpus(pd.load(texts_prior3)) ...]

all_docs=pd.append(taskpriors,processed_corpus)
words=ldamodel.train(all_docs).get_words.only_columns

prior_words=[]
for task in taskpriors:
    prior_words.append(ldamodel.train(task).get_words)

    
#These steps will be annoying.
prior_words=prior_words.fuse_together
prior_words=prior_words.extend(words)
#in the end prior words will have standard probability for words not seen their prior and the value of the prior else

model=ldamodel.train(given_corpus, eta=prior_words)


#From https://markroxor.github.io/gensim/static/notebooks/lda_training_tips.html
    def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
        # Init output
        sent_topics_df = pd.DataFrame()

        # Get main topic in each document
        for i, row in enumerate(ldamodel[corpus]):
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

        # Add original text to the end of the output
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
        return(sent_topics_df)


    df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=corpus, texts=data)

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()



    sent_topics_sorteddf = pd.DataFrame()

    sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, 
                                                 grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                                axis=0)

    # Reset Index    
    sent_topics_sorteddf.reset_index(drop=True, inplace=True)

    # Format
    sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

    # Show
    sent_topics_sorteddf.head()
